In [1]:
from urslib2 import RSS, DSSR
import os

In [1]:
# Create models from files

In [6]:

# Data directory, have to contain two sub-directories - models with cif-files and out with out-files 
mmcif    = '/home/baulin/Desktop/urs2/data/'

# Two example filenames
pdb = '1ffk.cif1'
out = '1ffk.out1'

# Path + filename
pdbmodel = mmcif+'models/'+pdb
outmodel = mmcif+'out/'+out

# If required cif-file exists:
if os.path.exists(pdbmodel):
    
    # Create out-file if it does not exist
    if not os.path.exists(outmodel):
        
        DSSR.run(pdbmodel, mmcif+'out/')
        
    # Create the structure object from the two files
    model = RSS.SecStruct(pdbmodel,outmodel)


In [2]:
from urslib2.Rfam import GetRfamInfo

In [3]:
help(GetRfamInfo)

Help on function GetRfamInfo in module urslib2.Rfam:

GetRfamInfo(path_to_rfam_files='', update=False)
    Parameters:
        path_to_rfam_files - path to family.txt and pdb_full_region.txt (by default - working directory)
        update - True/False - download new family.txt and pdb_full_region.txt
    Returns:
        dictionary {PDB-ID: {CHAIN: RFAM_INFO}}



In [4]:
GetRfamInfo()

Beginning Rfam download with urllib...
Unpacking...
Complete.


{}